## 1) Imports & configuration

We import geospatial and visualization libraries and set basic paths.


In [1]:
# === IMPORTY ===
from shapely.geometry import box, shape, mapping
from pystac_client import Client
import odc.stac
import pyproj
import rioxarray
import xarray as xr
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.features import shapes
import os

import leafmap.maplibregl as leafmap

## 2) Define AOI and time range

Adjust the AOI and the analysis window as needed.


In [2]:
# === PARAMETRE ===
STAC_URL = "https://stac.eodc.eu/api/v1"
COLLECTION_ID = "GFM"

aoi_geometry = box(74.162312, 32.098403, 74.601359, 32.174052)  # Pakistan AOI
time_range = ("2025-06-27", "2025-07-17")

print("AOI bounds:", aoi_geometry.bounds)
print("Time range:", time_range)

AOI bounds: (74.162312, 32.098403, 74.601359, 32.174052)
Time range: ('2025-06-27', '2025-07-17')


In [3]:
# === STAC QUERY ===
client = Client.open(STAC_URL)
search = client.search(
    collections=[COLLECTION_ID],
    intersects=mapping(aoi_geometry),
    datetime=f"{time_range[0]}/{time_range[1]}"
)
items = search.item_collection()
print(f"🔍 Found {len(items)} items.")
if len(items) == 0:
    raise RuntimeError("No STAC items found.")

# CRS & resolution
crs = pyproj.CRS.from_wkt(items[0].properties["proj:wkt2"])
res = items[0].properties.get("gsd", 20.0)
print("Source CRS:", crs)
print("Resolution:", res)

# === LOAD DATA ===
xx = odc.stac.load(
    items,
    crs=crs,
    bbox=aoi_geometry.bounds,
    bands=["ensemble_flood_extent"],
    resolution=res,
    dtype="uint8",
    fail_on_error=False,
)
xx = xx.rio.reproject("EPSG:4326")

🔍 Found 29 items.
Source CRS: PROJCS["Azimuthal_Equidistant",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]],PROJECTION["Azimuthal_Equidistant"],PARAMETER["false_easting",4340913.84808],PARAMETER["false_northing",4812712.92347],PARAMETER["longitude_of_center",94.0],PARAMETER["latitude_of_center",47.0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]
Resolution: 20.0


In [4]:
# === FLOOD DAYS → GeoJSON s atribútom days_flooded ===
import numpy as np
import geopandas as gpd
from shapely.geometry import shape
from rasterio.features import shapes

# 1) zdroj
flood = xx["ensemble_flood_extent"]  # (time, y, x), uint8, nodata=255 (podľa tvojho debug)

# 2) odmaskuj NoData a sprav bool (zaplavene/ne)
nodata = flood.rio.nodata
flood_masked = flood.where(flood != nodata) if nodata is not None else flood
flood_bool = (flood_masked > 0)

# 3) počet dní (sum cez čas)
flood_days = flood_bool.sum(dim="time").astype("uint16")   # (y, x), 0..T

# 4) numpy pole pre shapes()
arr = flood_days.data
if hasattr(arr, "compute"):  # dask -> numpy
    arr = arr.compute()
arr = np.ascontiguousarray(arr)

# 5) polygonizácia len tam, kde dni > 0 (8-susednosť)
transform = flood_days.rio.transform()
mask = arr > 0
geoms = shapes(arr, mask=mask, transform=transform, connectivity=8)

# 6) build GeoDataFrame s atribútom days_flooded
records = [{"geometry": shape(geom), "days_flooded": int(val)} for geom, val in geoms if val > 0]
gdf = gpd.GeoDataFrame(records, geometry="geometry", crs="EPSG:4326")

# (voliteľné) malé zjednodušenie podľa rozlíšenia, aby bol súbor menší
try:
    res_x, res_y = map(abs, flood_days.rio.resolution())
    tol = 0.5 * min(res_x, res_y)
except Exception:
    pass

# 7) export
out_file = "flood_days.geojson"
gdf.to_file(out_file, driver="GeoJSON", COORDINATE_PRECISION=6)
print(f"✅ Saved {out_file} | features: {len(gdf)}")

# rýchla kontrola rozdelenia hodnôt (koľko polí s 1 dňom, 2 dňami, …)
print(gdf["days_flooded"].value_counts().sort_index().head(20))


✅ Saved flood_days.geojson | features: 16950
days_flooded
1    12734
2     2647
3     1111
4      387
5       71
Name: count, dtype: int64


In [5]:
from IPython.display import HTML
HTML("""
<style>
.maplibregl-popup-content { 
  color: #111 !important; 
  background: #fff !important; 
}
.maplibregl-popup-tip { 
  border-top-color: #fff !important; 
}
.maplibregl-popup-close-button { 
  color: #111 !important;
}
</style>
""")


In [17]:
MAPTILER_KEY = "OT4sLDzLtErb4THhC5yl"

import leafmap.maplibregl as leafmap

m = leafmap.Map(
    pitch=45,
    bearing=0,
    style=f"https://api.maptiler.com/maps/streets-v2/style.json?key={MAPTILER_KEY}",
    #style=f"https://api.maptiler.com/maps/satellite/style.json?key={MAPTILER_KEY}",
)

url = "flood_days.geojson"

# 1 m na 1 deň
paint_fill = {
    "fill-extrusion-color": [
        "interpolate", ["linear"], ["to-number", ["get", "days_flooded"]],
        1, "#9ecae1",
        5, "#6baed6",
        10, "#4292c6",
        15, "#2171b5",
        20, "#08519c"
    ],
    "fill-extrusion-height": ["to-number", ["get", "days_flooded"]],  # ← 1 m / deň
    "fill-extrusion-base": 0,
    "fill-extrusion-opacity": 0.75,
}

m.add_geojson(url, layer_type="fill-extrusion", paint=paint_fill, name="flood-extent", fit_bounds=True)
m.add_popup("flood-extent")  # zobrazí properties (vrátane days_flooded)


m.to_html("map.html", title="Flood days 3D")


In [14]:
m.layer_interact()

In [9]:
MAPTILER_KEY = "OT4sLDzLtErb4THhC5yl"

import json, math
import leafmap.maplibregl as leafmap

# --- 1) Načítaj GeoJSON a vlož ho INLINE ---
with open("flood_days.geojson", "r", encoding="utf-8") as f:
    fc = json.load(f)
assert fc.get("features"), "GeoJSON je prázdny."

# --- 2) Mapa (bez vektorového štýlu) ---
m = leafmap.Map(pitch=45, bearing=0, center=[0,0], zoom=2)

# Podklad RASTER (vyber si JEDEN riadok a druhý nechaj zakomentovaný)

# a) OSM (bez kľúča)
m.add_tile_layer("https://tile.openstreetmap.org/{z}/{x}/{y}.png",
                 name="OSM", opacity=1.0, attribution="© OpenStreetMap")

# b) MapTiler raster (s kľúčom) – ak chceš satelit:
# m.add_tile_layer(f"https://api.maptiler.com/tiles/satellite-v2/{{z}}/{{x}}/{{y}}.jpg?key={MAPTILER_KEY}",
#                  name="Satellite", opacity=1.0)

# --- 3) Zdroj s vloženými dátami + 3D extrúzia (1 m / 1 deň) ---
m.add_source("flood-src", {"type": "geojson", "data": fc})

paint_fill = {
    "fill-extrusion-color": [
        "interpolate", ["linear"], ["to-number", ["coalesce", ["get", "days_flooded"], 0]],
        0, "#f0f9ff",
        1, "#9ecae1",
        5, "#6baed6",
        10, "#4292c6",
        15, "#2171b5",
        20, "#08519c",
    ],
    "fill-extrusion-height": ["to-number", ["coalesce", ["get", "days_flooded"], 0]],
    "fill-extrusion-opacity": 0.75,
}

m.add_layer({
    "id": "flood-3d",
    "type": "fill-extrusion",
    "source": "flood-src",
    "paint": paint_fill,
}, name="flood-3d")

# (voliteľné) presný popup (cez 2D hit vrstvu, aby klik vždy ukázal správne hodnoty)
m.add_layer({"id": "flood-hit", "type": "fill", "source": "flood-src",
             "paint": {"fill-opacity": 0.0}}, name="flood-hit")
m.add_popup("flood-hit")

# --- 4) Spoľahlivé fit_bounds z dát ---
def fc_bounds(fc):
    minx, miny, maxx, maxy = math.inf, math.inf, -math.inf, -math.inf
    for ft in fc["features"]:
        g = ft.get("geometry", {})
        if g.get("type") == "Polygon":
            for ring in g["coordinates"]:
                for x, y in ring:
                    minx = x if x < minx else minx
                    miny = y if y < miny else miny
                    maxx = x if x > maxx else maxx
                    maxy = y if y > maxy else maxy
        elif g.get("type") == "MultiPolygon":
            for poly in g["coordinates"]:
                for ring in poly:
                    for x, y in ring:
                        minx = x if x < minx else minx
                        miny = y if y < miny else miny
                        maxx = x if x > maxx else maxx
                        maxy = y if y > maxy else maxy
    return [[min(ys := [miny, maxy]), min(xs := [minx, maxx])],
            [max(ys), max(xs)]]

m.fit_bounds(fc_bounds(fc))

# --- 5) Export – HTML obsahuje tvoje dáta priamo v sebe ---
m.to_html("map.html", title="Flood days 3D (inline)")
print("✅ map.html hotové")


The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


Container(children=[Row(children=[Col(children=[Col(children=[Map(calls=[['addControl', ('NavigationControl', …